In [7]:
# Torch geometric modules
from torch_geometric.data import HeteroData, DataLoader
import torch_geometric.transforms as T
from torch_geometric.nn import to_hetero , HeteroConv , GATv2Conv
from torch_geometric.utils import negative_sampling
from torch_geometric.loader import LinkNeighborLoader

# Torch modules
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

# SKlearn modules
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder , label_binarize , OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score , matthews_corrcoef

# Ground modules
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from itertools import product
import random
from collections import Counter
import warnings
import logging
from multiprocessing.pool import ThreadPool

# TropiGAT modules
import TropiGAT_graph
import TropiGAT_models

warnings.filterwarnings("ignore")

# *****************************************************************************
# Load the Dataframes :
#path_work = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023/train_nn"
path_work = "/media/concha-eloko/Linux/PPT_clean"

DF_info = pd.read_csv(f"{path_work}/TropiGATv2.final_df_v2.tsv", sep = "\t" ,  header = 0)
DF_info = DF_info.drop_duplicates(subset = ["Protein_name"])
DF_info = DF_info[~DF_info["KL_type_LCA"].str.contains("\\|")]

df_prophages = DF_info.drop_duplicates(subset = ["Phage"], keep = "first")
dico_prophage_info = {row["Phage"] : {"prophage_strain" : row["prophage_id"] , "ancestor" : row["Infected_ancestor"], "KL_type" : row["KL_type_LCA"]} for _,row in df_prophages.iterrows()}


In [6]:
def get_filtered_prophages(prophage) :
    combinations = []
    to_exclude = set()
    to_keep = set()
    to_keep.add(prophage)
    df_prophage_group = DF_info[(DF_info["prophage_id"] == dico_prophage_info[prophage]["prophage_strain"]) & (DF_info["Infected_ancestor"] == dico_prophage_info[prophage]["ancestor"])]
    if len(df_prophage_group) == 1 :
        pass
    else :
        depo_set = set(df_prophage_group[df_prophage_group["Phage"] == prophage]["domain_seq"].values)
        for prophage_tmp in df_prophage_group["Phage"].unique().tolist() :
            if prophage_tmp != prophage :
                tmp_depo_set = set(df_prophage_group[df_prophage_group["Phage"] == prophage_tmp]["domain_seq"].values)
                if depo_set == tmp_depo_set :
                    to_exclude.add(prophage_tmp)
                else :
                    if tmp_depo_set not in combinations :
                        to_keep.add(prophage_tmp)
                        combinations.append(tmp_depo_set)
                    else :
                        to_exclude.add(prophage_tmp)
    return df_prophage_group , to_exclude , to_keep

good_prophages = set()
excluded_prophages = set()

for prophage, info_prophage in tqdm(dico_prophage_info.items()) :
    if prophage not in excluded_prophages and prophage not in good_prophages:
        _, excluded_members , kept_members = get_filtered_prophages(prophage)
        good_prophages.update(kept_members)
        excluded_prophages.update(excluded_members)

DF_info_lvl_0_filtered = DF_info[DF_info["Phage"].isin(good_prophages)]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15218/15218 [01:06<00:00, 230.45it/s]


In [13]:
DF_info_lvl_0_filtered

,Phage,Protein_name,KL_type_LCA,Infected_ancestor,index,Dataset,seq,domain_seq,1,2,...,1272,1273,1274,1275,1276,1277,1278,1279,1280,prophage_id
0,GCF_902164905.1__phage1,GCF_902164905.1__phage1__34,KL41,GCF_902164905.1,minibatch__460,minibatch,MPATPQDRLYGLTTSVAVKPPVFISVDYDVARFGEQTITSKTPTDE...,QDRLYGLTTSVAVKPPVFISVDYDVARFGEQTITSKTPTDERTITT...,0.025276,0.053137,...,-0.011464,0.081105,0.012011,0.042917,0.009402,0.093175,-0.080562,0.000897,0.111854,prophage_11309
4,GCF_017310305.1__phage5,GCF_017310305.1__phage5__1353,KL30,n4996,minibatch__1084,minibatch,MTVSTQVSRNEYTGNGATTQYDFTFRILDKSHLLVQTMDTSENIVT...,VSTQVSRNEYTGNGATTQYDFTFRILDKSHLLVQTMDTSENIVTLT...,0.004905,0.040896,...,-0.040657,0.087288,0.022292,0.024434,0.025246,0.083449,-0.123537,0.047648,0.061250,prophage_5
10,GCF_001701985.1__phage2,GCF_001701985.1__phage2__357,KL30,n4988,minibatch__1084,minibatch,MTVSTQVSRNEYTGNGATTQYDFTFRILDKSHLLVQTMDTSENIVT...,VSTQVSRNEYTGNGATTQYDFTFRILDKSHLLVQTMDTSENIVTLT...,0.004905,0.040896,...,-0.040657,0.087288,0.022292,0.024434,0.025246,0.083449,-0.123537,0.047648,0.061250,prophage_6465
12,GCF_001611095.1__phage5,GCF_001611095.1__phage5__1365,KL30,n49894989,minibatch__1084,minibatch,MTVSTQVSRNEYTGNGATTQYDFTFRILDKSHLLVQTMDTSENIVT...,VSTQVSRNEYTGNGATTQYDFTFRILDKSHLLVQTMDTSENIVTLT...,0.004905,0.040896,...,-0.040657,0.087288,0.022292,0.024434,0.025246,0.083449,-0.123537,0.047648,0.061250,prophage_5
15,GCF_902156555.1__phage3,GCF_902156555.1__phage3__511,KL30,GCF_902156555.1,minibatch__1084,minibatch,MTVSTQVSRNEYTGNGATTQYDFTFRILDKSHLLVQTMDTSENIVT...,VSTQVSRNEYTGNGATTQYDFTFRILDKSHLLVQTMDTSENIVTLT...,0.004905,0.040896,...,-0.040657,0.087288,0.022292,0.024434,0.025246,0.083449,-0.123537,0.047648,0.061250,prophage_1828
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21342,GCF_900506765.1__phage17,GCF_900506765.1__phage17__90,KL149,GCF_900506765.1,anubis_return__4216,anubis_return,MMTTLNEHPQWESDIYLIKRSDLVAGGRGGIANMQAQQLANRTAFL...,NRRWFRRFTGNIRAEWSGIHDLSQSSAPVDSYIYRLLLASAVGSPD...,0.053704,0.083858,...,0.032803,0.109572,0.010032,0.024949,0.094129,0.028693,-0.061396,0.006824,0.046220,prophage_15598
21344,GCF_003255785.1__phage1,GCF_003255785.1__phage1__10,KL127,GCF_003255785.1,anubis_return__4239,anubis_return,MNGLNHNALTCSAVPIPPWERSLQTVEAQPYFSVSQASLVLEGIVF...,MNGLNHNALTCSAVPIPPWERSLQTVEAQPYFSVSQASLVLEGIVF...,0.010626,-0.025389,...,0.045372,0.009262,-0.008319,-0.050856,0.034115,0.101663,-0.108278,-0.135629,0.102486,prophage_3577
21346,GCF_002186895.1__phage9,GCF_002186895.1__phage9__5,KL57,GCF_002186895.1,anubis_return__4260,anubis_return,MRYRFIALALCLLSGSKVAISAGFDCSLANLSPTEKTICSNEYLSG...,ITDSPWLVKKIFSSDSFEGGINLEGMNVSSILTYQEIKNDLYIYIS...,0.073450,0.046651,...,0.035302,0.012151,0.003563,-0.022575,0.014130,0.063376,-0.050646,-0.085156,-0.010849,prophage_6002
21347,GCF_004312845.1__phage3,GCF_004312845.1__phage3__38,KL9,GCF_004312845.1,anubis_return__4275,anubis_return,MAILITGKSMTRLPESSSWEEEIELITRSERVAGGLDGPANRPLKS...,DAVIRRDLASDKGTSGVGKLGDKPLVAISYYKSKGQSDQDAVQAAF...,0.032196,0.048856,...,-0.016331,0.084711,0.056063,0.001793,0.073958,0.090169,-0.060105,0.023726,0.086452,prophage_12656


***
## Check for duplicates : 

There are duplicate sets of depolymerases, encoded by prophages. We argue that they follow the natural distribution and therefore allow for weighting the importance of depolymerase.

Then what we want is to finetune esm2 models.

In [9]:
duplicate_prophage = []
dico_kltype_duplica = {}
for kltype in DF_info_lvl_0_filtered["KL_type_LCA"].unique():
    df_kl = DF_info_lvl_0_filtered[DF_info_lvl_0_filtered["KL_type_LCA"] == kltype][["Phage", "Protein_name", "KL_type_LCA", "Infected_ancestor", "index", "seq", "domain_seq"]]
    prophages_tmp_list = df_kl["Phage"].unique().tolist()
    set_sets_depo = []
    duplicated = {}  
    for prophage_tmp in prophages_tmp_list: 
        set_depo = frozenset(df_kl[df_kl["Phage"] == prophage_tmp]["domain_seq"].values)
        for past_set in set_sets_depo:
            if past_set == set_depo:
                duplicated[past_set] = duplicated.get(past_set, 0) + 1
                duplicate_prophage.append(prophage_tmp)
                break
        else:
            set_sets_depo.append(set_depo)
            duplicated[set_depo] = 1
    dico_kltype_duplica[kltype] = duplicated



In [47]:
for kltype in dico_kltype_duplica :
    print(kltype, dico_kltype_duplica[kltype].values(), len(dico_kltype_duplica[kltype].values()), sep = "\n")
    print("\n")

KL41
dict_values([1, 1, 1, 1, 1, 1, 1, 1, 2])
9


KL30
dict_values([3, 3, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 7, 3, 1, 1, 1, 2, 1, 1, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 2, 1, 1, 1, 2, 1, 1, 2, 3, 1])
63


KL6
dict_values([1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1])
11


KL19
dict_values([6, 1, 2, 1, 4, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 12, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
45


KL25
dict_values([4, 1, 2, 1, 1, 1, 1, 1, 1, 1, 4, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 1, 5, 5, 1, 1, 1, 3, 10, 1, 2, 7, 10, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 2, 1, 1, 4, 1, 1, 1, 4, 2, 1, 1, 2, 1, 1, 2, 3, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 6, 1, 1, 8, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 14, 1, 3, 7, 1, 1, 1, 1, 1, 1, 1, 23, 1, 3, 1, 3, 1, 1])
141


KL123
dict_values([2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [10]:
DF_info_lvl_0_ultra_filtered = DF_info_lvl_0_filtered[~DF_info_lvl_0_filtered["Phage"].isin(duplicate_prophage)]
DF_info_lvl_0_ultra_filtered

,Phage,Protein_name,KL_type_LCA,Infected_ancestor,index,Dataset,seq,domain_seq,1,2,...,1272,1273,1274,1275,1276,1277,1278,1279,1280,prophage_id
0,GCF_902164905.1__phage1,GCF_902164905.1__phage1__34,KL41,GCF_902164905.1,minibatch__460,minibatch,MPATPQDRLYGLTTSVAVKPPVFISVDYDVARFGEQTITSKTPTDE...,QDRLYGLTTSVAVKPPVFISVDYDVARFGEQTITSKTPTDERTITT...,0.025276,0.053137,...,-0.011464,0.081105,0.012011,0.042917,0.009402,0.093175,-0.080562,0.000897,0.111854,prophage_11309
4,GCF_017310305.1__phage5,GCF_017310305.1__phage5__1353,KL30,n4996,minibatch__1084,minibatch,MTVSTQVSRNEYTGNGATTQYDFTFRILDKSHLLVQTMDTSENIVT...,VSTQVSRNEYTGNGATTQYDFTFRILDKSHLLVQTMDTSENIVTLT...,0.004905,0.040896,...,-0.040657,0.087288,0.022292,0.024434,0.025246,0.083449,-0.123537,0.047648,0.061250,prophage_5
10,GCF_001701985.1__phage2,GCF_001701985.1__phage2__357,KL30,n4988,minibatch__1084,minibatch,MTVSTQVSRNEYTGNGATTQYDFTFRILDKSHLLVQTMDTSENIVT...,VSTQVSRNEYTGNGATTQYDFTFRILDKSHLLVQTMDTSENIVTLT...,0.004905,0.040896,...,-0.040657,0.087288,0.022292,0.024434,0.025246,0.083449,-0.123537,0.047648,0.061250,prophage_6465
21,GCF_900622625.1__phage2,GCF_900622625.1__phage2__2892,KL6,GCF_900622625.1,minibatch__1741,minibatch,MAFNPELGSSSPEVLLDNAKRLDELTNGPAATVPDRAGEPLDSWRK...,ELGSSSPEVLLDNAKRLDELTNGPAATVPDRAGEPLDSWRKMQEDN...,-0.003624,-0.032987,...,-0.075218,-0.010726,0.008995,-0.001741,-0.103979,0.119088,-0.124593,0.011745,0.147883,prophage_4098
22,GCF_011044795.1__phage17,GCF_011044795.1__phage17__11,KL19,80.7/1001331,minibatch__467,minibatch,MNRSRRLLMRGIGYLTLFPLLFLFSKKVSSAPNGLTEKVKNRKIEK...,RSRRLLMRGIGYLTLFPLLFLFSKKVSSAPNGLTEKVKNRKIEKDV...,0.038219,0.037305,...,0.010699,0.013225,0.038260,-0.001471,0.040612,0.066368,-0.078655,0.031434,0.080821,prophage_4997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21342,GCF_900506765.1__phage17,GCF_900506765.1__phage17__90,KL149,GCF_900506765.1,anubis_return__4216,anubis_return,MMTTLNEHPQWESDIYLIKRSDLVAGGRGGIANMQAQQLANRTAFL...,NRRWFRRFTGNIRAEWSGIHDLSQSSAPVDSYIYRLLLASAVGSPD...,0.053704,0.083858,...,0.032803,0.109572,0.010032,0.024949,0.094129,0.028693,-0.061396,0.006824,0.046220,prophage_15598
21344,GCF_003255785.1__phage1,GCF_003255785.1__phage1__10,KL127,GCF_003255785.1,anubis_return__4239,anubis_return,MNGLNHNALTCSAVPIPPWERSLQTVEAQPYFSVSQASLVLEGIVF...,MNGLNHNALTCSAVPIPPWERSLQTVEAQPYFSVSQASLVLEGIVF...,0.010626,-0.025389,...,0.045372,0.009262,-0.008319,-0.050856,0.034115,0.101663,-0.108278,-0.135629,0.102486,prophage_3577
21346,GCF_002186895.1__phage9,GCF_002186895.1__phage9__5,KL57,GCF_002186895.1,anubis_return__4260,anubis_return,MRYRFIALALCLLSGSKVAISAGFDCSLANLSPTEKTICSNEYLSG...,ITDSPWLVKKIFSSDSFEGGINLEGMNVSSILTYQEIKNDLYIYIS...,0.073450,0.046651,...,0.035302,0.012151,0.003563,-0.022575,0.014130,0.063376,-0.050646,-0.085156,-0.010849,prophage_6002
21347,GCF_004312845.1__phage3,GCF_004312845.1__phage3__38,KL9,GCF_004312845.1,anubis_return__4275,anubis_return,MAILITGKSMTRLPESSSWEEEIELITRSERVAGGLDGPANRPLKS...,DAVIRRDLASDKGTSGVGKLGDKPLVAISYYKSKGQSDQDAVQAAF...,0.032196,0.048856,...,-0.016331,0.084711,0.056063,0.001793,0.073958,0.090169,-0.060105,0.023726,0.086452,prophage_12656


In [14]:
DF_info_lvl_0_ultra_filtered[DF_info_lvl_0_ultra_filtered["KL_type_LCA"] == "KL64"].drop_duplicates(subset = ["Phage"])

,Phage,Protein_name,KL_type_LCA,Infected_ancestor,index,Dataset,seq,domain_seq,1,2,...,1272,1273,1274,1275,1276,1277,1278,1279,1280,prophage_id
57,GCF_020177805.1__phage25,GCF_020177805.1__phage25__109,KL64,n27912791,minibatch__804,minibatch,MAEVPLPTPTQVPVPSTDIRNAVFAGAKLDEEVTGTGEFYTDRLGA...,VPLPTPTQVPVPSTDIRNAVFAGAKLDEEVTGTGEFYTDRLGAKRL...,0.019308,0.008050,...,-0.053787,0.058374,0.033429,0.029935,-0.012611,0.062793,-0.150896,0.061797,0.108786,prophage_5502
159,GCF_904863365.1__phage6,GCF_904863365.1__phage6__3592,KL64,n17081708,minibatch__271,minibatch,MAEVPLPTPTQAPVPSTDIRNAVFAGAKLDEEVTGTGEFYTDRLGV...,EVPLPTPTQAPVPSTDIRNAVFAGAKLDEEVTGTGEFYTDRLGVKR...,0.034148,0.016124,...,-0.045861,0.077156,0.021072,-0.003311,-0.020224,0.094309,-0.122477,0.038291,0.121914,prophage_1147
171,GCF_003073335.1__phage14,GCF_003073335.1__phage14__4345,KL64,GCF_003073335.1,minibatch__2079,minibatch,MTVSTEVDHNDYTGNGVTTSFPYTFRIFKKSDLTVQLADLNENITV...,TEVDHNDYTGNGVTTSFPYTFRIFKKSDLTVQLADLNENITVLTLD...,-0.003399,0.010523,...,-0.036729,0.055501,0.023223,0.028232,0.011881,0.082231,-0.135116,0.032867,0.081216,prophage_4523
210,GCF_900509485.1__phage10,GCF_900509485.1__phage10__27,KL64,GCF_900509485.1,minibatch__659,minibatch,MTVSTEVDHNDYTGNGVTTSFPYTFRIFKKSDLTVQLADLNENITV...,TEVDHNDYTGNGVTTSFPYTFRIFKKSDLTVQLADLNENITVLTLD...,-0.004660,0.008725,...,-0.033908,0.057584,0.024042,0.025956,0.009719,0.080979,-0.131810,0.034406,0.078299,prophage_9977
216,GCF_019444195.1__phage4,GCF_019444195.1__phage4__1393,KL64,n2673,minibatch__828,minibatch,MTVSTQVSRNEYTGNGATTQYDFTFRILDKSHLLVQTMDTSENIVT...,TQVSRNEYTGNGATTQYDFTFRILDKSHLLVQTMDTSENIVTLTLG...,0.003391,0.030314,...,-0.050629,0.063792,0.012182,0.038767,0.012858,0.058645,-0.140730,0.047510,0.074003,prophage_11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20569,GCF_019445415.1__phage23,GCF_019445415.1__phage23__0,KL64,n27032703,anubis_return__809,anubis_return,MASIKELPRWEDEVYQIARGDKVEGGVGGIANMQAKTLAERTRYLK...,DPSAPEPMRMFNIKGKPRIELTAEELATFNATYSKYLKKGSQYLPY...,0.014498,0.081194,...,-0.018137,0.036657,0.069563,0.021782,0.182507,0.101342,-0.075100,0.029229,-0.054190,prophage_12
20630,GCF_905128575.1__phage7,GCF_905128575.1__phage7__28,KL64,93.2/100,anubis_return__898,anubis_return,MKQKSLYIAIVLALHCAVAVADNSIDSTQIDEVTVNGDGNNISGLT...,LHCAVAVADNSIDSTQIDEVTVNGDGNNISGLTPCATPSNCKSPLG...,0.015749,0.062796,...,0.061557,0.092773,-0.039177,0.006807,0.010053,0.010442,-0.063088,-0.053337,0.035094,prophage_2239
21026,GCF_019433735.1__phage7,GCF_019433735.1__phage7__18,KL64,GCF_019433735.1,anubis_return__2337,anubis_return,MNGLNHNALTCSAVPIPPWERPLQTVEAQPYFNVSQASLVLEGIVF...,MNGLNHNALTCSAVPIPPWERPLQTVEAQPYFNVSQASLVLEGIVF...,0.008892,-0.023124,...,0.043436,0.007463,-0.012128,-0.047250,0.033145,0.101437,-0.113360,-0.130255,0.107618,prophage_15558
21236,GCF_002262805.1__phage9,GCF_002262805.1__phage9__0,KL64,n2841,anubis_return__3411,anubis_return,MLKHSLAIATCLAFSSSVMGNEANLLYTNTMQFPYKHNADGYMVFD...,AFSSSVMGNEANLLYTNTMQFPYKHNADGYMVFDIHGKLVVPPEGH...,0.058785,0.041078,...,0.063646,-0.019397,0.004691,0.001815,-0.015139,0.070408,-0.044197,-0.073100,-0.074167,prophage_313


> On the server: 

In [ ]:
duplicate_prophage = []
dico_kltype_duplica = {}
for kltype in DF_info_lvl_0_final["KL_type_LCA"].unique():
    df_kl = DF_info_lvl_0_final[DF_info_lvl_0_final["KL_type_LCA"] == kltype][["Phage", "Protein_name", "KL_type_LCA", "Infected_ancestor", "index", "seq", "domain_seq"]]
    prophages_tmp_list = df_kl["Phage"].unique().tolist()
    set_sets_depo = []
    duplicated = {}  
    for prophage_tmp in prophages_tmp_list: 
        set_depo = frozenset(df_kl[df_kl["Phage"] == prophage_tmp]["domain_seq"].values)
        for past_set in set_sets_depo:
            if past_set == set_depo:
                duplicated[past_set] = duplicated.get(past_set, 0) + 1
                duplicate_prophage.append(prophage_tmp)
                break
        else:
            set_sets_depo.append(set_depo)
            duplicated[set_depo] = 1
    dico_kltype_duplica[kltype] = duplicated
    

DF_info_lvl_0_final_ultrafiltered = DF_info_lvl_0_final[~DF_info_lvl_0_final["Phage"].isin(duplicate_prophage)]
    


***
### Next :

In [ ]:
def get_filtered_prophages(prophage) :
    combinations = []
    to_exclude = set()
    to_keep = set()
    to_keep.add(prophage)
    df_prophage_group = DF_info[(DF_info["prophage_id"] == dico_prophage_info[prophage]["prophage_strain"]) & (DF_info["Infected_ancestor"] == dico_prophage_info[prophage]["ancestor"])]
    if len(df_prophage_group) == 1 :
        pass
    else :
        depo_set = set(df_prophage_group[df_prophage_group["Phage"] == prophage]["domain_seq"].values)
        for prophage_tmp in df_prophage_group["Phage"].unique().tolist() :
            if prophage_tmp != prophage :
                tmp_depo_set = set(df_prophage_group[df_prophage_group["Phage"] == prophage_tmp]["domain_seq"].values)
                if depo_set == tmp_depo_set :
                    to_exclude.add(prophage_tmp)
                else :
                    if tmp_depo_set not in combinations :
                        to_keep.add(prophage_tmp)
                        combinations.append(tmp_depo_set)
                    else :
                        to_exclude.add(prophage_tmp)
    return df_prophage_group , to_exclude , to_keep

good_prophages = set()
excluded_prophages = set()

for prophage, info_prophage in tqdm(dico_prophage_info.items()) :
    if prophage not in excluded_prophages and prophage not in good_prophages:
        _, excluded_members , kept_members = get_filtered_prophages(prophage)
        good_prophages.update(kept_members)
        excluded_prophages.update(excluded_members)

DF_info_lvl_0_filtered = DF_info[DF_info["Phage"].isin(good_prophages)]
DF_info_lvl_0_final = DF_info_lvl_0_filtered[~DF_info_lvl_0_filtered["KL_type_LCA"].str.contains("\\|")]


# Ultrafiltration
duplicate_prophage = []
dico_kltype_duplica = {}
for kltype in DF_info_lvl_0_final["KL_type_LCA"].unique():
    df_kl = DF_info_lvl_0_final[DF_info_lvl_0_final["KL_type_LCA"] == kltype][["Phage", "Protein_name", "KL_type_LCA", "Infected_ancestor", "index", "seq", "domain_seq"]]
    prophages_tmp_list = df_kl["Phage"].unique().tolist()
    set_sets_depo = []
    duplicated = {}  
    for prophage_tmp in prophages_tmp_list: 
        set_depo = frozenset(df_kl[df_kl["Phage"] == prophage_tmp]["domain_seq"].values)
        for past_set in set_sets_depo:
            if past_set == set_depo:
                duplicated[past_set] = duplicated.get(past_set, 0) + 1
                duplicate_prophage.append(prophage_tmp)
                break
        else:
            set_sets_depo.append(set_depo)
            duplicated[set_depo] = 1
    dico_kltype_duplica[kltype] = duplicated
    

DF_info_lvl_0_final_ultrafiltered = DF_info_lvl_0_final[~DF_info_lvl_0_final["Phage"].isin(duplicate_prophage)]
DF_info_lvl_0 = DF_info_lvl_0_final_ultrafiltered.copy()



# Log file :
path_ensemble = f"{path_work}/train_nn/ensemble_08092024"

os.makedirs(path_ensemble, exist_ok = True)

df_prophages = DF_info_lvl_0.drop_duplicates(subset = ["Phage"])
dico_prophage_count = dict(Counter(df_prophages["KL_type_LCA"]))

KLtypes = [kltype for kltype in dico_prophage_count if dico_prophage_count[kltype] >= 20]


# *****************************************************************************
# Make graphs :
graph_baseline , dico_prophage_kltype_associated = TropiGAT_graph.build_graph_baseline(DF_info_lvl_0)
#graph_dico = {kltype : TropiGAT_graph.build_graph_masking(graph_baseline , dico_prophage_kltype_associated,DF_info_lvl_0, kltype, 5, 1, 0, 0)
#             for kltype in DF_info_lvl_0["KL_type_LCA"].unique()}




# *****************************************************************************
def train_graph(KL_type) :
    for seed in range(1,6) :
        with open(f"{path_work}/train_nn/ensemble_08092024/{KL_type}__{seed}__node_classification.0809.log" , "w") as log_outfile :
            n_prophage = dico_prophage_count[KL_type]
            graph_data_kltype = TropiGAT_graph.build_graph_masking_v2(graph_baseline , dico_prophage_kltype_associated, DF_info_lvl_0, KL_type, 5, 0.7, 0.2, 0.1, seed = seed)
            if n_prophage <= 125 : 
                model = TropiGAT_models.TropiGAT_small_module(hidden_channels = 1280, heads = 1)
                n = "small"
            else : 
                #model = TropiGAT_models.TropiGAT_big_module(hidden_channels = 1280, heads = 1)
                model = TropiGAT_models.TropiGAT_small_module(hidden_channels = 1280, heads = 1)
                n = "small" #big before
            model(graph_data_kltype)
            optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001 , weight_decay= 0.000001)
            scheduler = ReduceLROnPlateau(optimizer, 'min')
            criterion = torch.nn.BCEWithLogitsLoss()
            early_stopping = TropiGAT_models.EarlyStopping(patience=60, verbose=True, path=f"{path_ensemble}/{KL_type}__{seed}.TropiGATv2.0809.pt", metric='MCC')
            try : 
                for epoch in range(200):
                    train_loss = TropiGAT_models.train(model, graph_data_kltype, optimizer,criterion)
                    if epoch % 5 == 0:
                        # Get all metrics
                        test_loss, metrics = TropiGAT_models.evaluate(model, graph_data_kltype,criterion, graph_data_kltype["B1"].test_mask)
                        info_training_concise = f'Epoch: {epoch}\tTrain Loss: {train_loss}\tTest Loss: {test_loss}\tMCC: {metrics[3]}\tAUC: {metrics[5]}\tAccuracy: {metrics[4]}\n'
                        info_training = f'Epoch: {epoch}, Train Loss: {train_loss}, Test Loss: {test_loss},F1 Score: {metrics[0]}, Precision: {metrics[1]}, Recall: {metrics[2]}, MCC: {metrics[3]},Accuracy: {metrics[4]}, AUC: {metrics[5]}'
                        log_outfile.write(info_training_concise)
                        scheduler.step(test_loss)
                        early_stopping(metrics[3], model, epoch)
                        if early_stopping.early_stop:
                            log_outfile.write(f"Early stopping at epoch = {epoch}\n")
                            break
                else :
                    torch.save(model, f"{path_ensemble}/{KL_type}__{seed}.TropiGATv2.0809.pt")
                # The final eval :
                model_final = TropiGAT_models.TropiGAT_small_module(hidden_channels = 1280, heads = 1)
                model_final.load_state_dict(torch.load(f"{path_ensemble}/{KL_type}__{seed}.TropiGATv2.0809.pt"))
                eval_loss, metrics = TropiGAT_models.evaluate(model_final, graph_data_kltype, criterion,graph_data_kltype["B1"].eval_mask)
                with open(f"{path_ensemble}/Metric_Report.0809.tsv", "a+") as metric_outfile :
                    metric_outfile.write(f"{KL_type}__{seed}\t{n_prophage}\t{metrics[0]}\t{metrics[1]}\t{metrics[2]}\t{metrics[3]}\t{metrics[4]}\t{metrics[5]}\n")
                info_eval = f'Epoch: {epoch}, F1 Score: {metrics[0]}, Precision: {metrics[1]}, Recall: {metrics[2]}, MCC: {metrics[3]},Accuracy: {metrics[4]}, AUC: {metrics[5]}'
                log_outfile.write(f"Final evaluation ...\n{info_eval}")
            except Exception as e :
                log_outfile.write(f"***Issue here : {e}")
                with open(f"{path_ensemble}/Metric_Report.0809.tsv", "a+") as metric_outfile :
                    n_prophage = dico_prophage_count[KL_type]
                    metric_outfile.write(f"{KL_type}__{seed}\t{n_prophage}\t***Issue***\n")

                    
                    
if __name__ == '__main__':
    with ThreadPool(5) as p:
        p.map(train_graph, KLtypes)


In [ ]:
def train_graph(KL_type):
    for seed in range(1, 6):
        log_file = f"{path_work}/train_nn/ensemble_08092024/{KL_type}__{seed}__node_classification.0809.log"
        with open(log_file, "w") as log_outfile:
            graph_data_kltype = TropiGAT_graph.build_graph_masking_v2(
                graph_baseline, dico_prophage_kltype_associated, DF_info_lvl_0,
                KL_type, 5, 0.7, 0.2, 0.1, seed=seed
            )

            model = TropiGAT_models.TropiGAT_small_module(hidden_channels=1280, heads=1)
            model(graph_data_kltype)
            optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.000001)
            scheduler = ReduceLROnPlateau(optimizer, 'min')
            criterion = torch.nn.BCEWithLogitsLoss()
            early_stopping = TropiGAT_models.EarlyStopping(
                patience=60, verbose=True,
                path=f"{path_ensemble}/{KL_type}__{seed}.TropiGATv2.0809.pt",
                metric='MCC'
            )

            try:
                for epoch in range(200):
                    train_loss = TropiGAT_models.train(model, graph_data_kltype, optimizer, criterion)

                    if epoch % 5 == 0:
                        test_loss, metrics = TropiGAT_models.evaluate(
                            model, graph_data_kltype, criterion, graph_data_kltype["B1"].test_mask
                        )
                        info_training_concise = (
                            f'Epoch: {epoch}\tTrain Loss: {train_loss}\tTest Loss: {test_loss}'
                            f'\tMCC: {metrics[3]}\tAUC: {metrics[5]}\tAccuracy: {metrics[4]}'
                        )
                        info_training = (
                            f'Epoch: {epoch}, Train Loss: {train_loss}, Test Loss: {test_loss},'
                            f'F1 Score: {metrics[0]}, Precision: {metrics[1]}, Recall: {metrics[2]},'
                            f'MCC: {metrics[3]}, Accuracy: {metrics[4]}, AUC: {metrics[5]}'
                        )
                        log_outfile.write(info_training_concise + "\n")
                        scheduler.step(test_loss)
                        early_stopping(metrics[3], model, epoch)
                        if early_stopping.early_stop:
                            log_outfile.write(f"Early stopping at epoch = {epoch}\n")
                            break
                else:
                    torch.save(model, f"{path_ensemble}/{KL_type}__{seed}.TropiGATv2.0809.pt")

                model_final = TropiGAT_models.TropiGAT_small_module(hidden_channels=1280, heads=1)
                model_final.load_state_dict(torch.load(f"{path_ensemble}/{KL_type}__{seed}.TropiGATv2.0809.pt"))
                eval_loss, metrics = TropiGAT_models.evaluate(
                    model_final, graph_data_kltype, criterion, graph_data_kltype["B1"].eval_mask
                )
                with open(f"{path_ensemble}/Metric_Report.0809.tsv", "a+") as metric_outfile:
                    metric_outfile.write(
                        f"{KL_type}__{seed}\t{dico_prophage_count[KL_type]}\t{metrics[0]}\t{metrics[1]}"
                        f"\t{metrics[2]}\t{metrics[3]}\t{metrics[4]}\t{metrics[5]}\n"
                    )
                info_eval = (
                    f'Epoch: {epoch}, F1 Score: {metrics[0]}, Precision: {metrics[1]},'
                    f'Recall: {metrics[2]}, MCC: {metrics[3]}, Accuracy: {metrics[4]}, AUC: {metrics[5]}'
                )
                log_outfile.write(f"Final evaluation ...\n{info_eval}")
            except Exception as e:
                log_outfile.write(f"***Issue here : {e}")
                with open(f"{path_ensemble}/Metric_Report.0809.tsv", "a+") as metric_outfile:
                    metric_outfile.write(f"{KL_type}__{seed}\t{dico_prophage_count[KL_type]}\t***Issue***\n")


> Move the files back 

In [ ]:
rsync -avzhe ssh \
conchae@garnatxa.srv.cpd:/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023/train_nn/ensemble_08092024 \
/media/concha-eloko/Linux/PPT_clean/ficheros_28032023

rsync -avzhe ssh \
conchae@garnatxa.srv.cpd:/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023/train_nn/ensemble_08092024_log_files \
/media/concha-eloko/Linux/PPT_clean/ficheros_28032023